## Advise re identity architecture

Initialize

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.4.0-preview"
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.4.0"

#r "nuget: System.Linq.Async, 6.0.1"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;

using Kernel = Microsoft.SemanticKernel.Kernel;

var (useAzureOpenAI, model, azureEndpoint, apiKey, bingApiKey, orgId) = Settings.LoadFromFile();
var kernel = Microsoft.SemanticKernel.Kernel.CreateBuilder()
            .AddAzureOpenAIChatCompletion(
                model,   // deployment name
                azureEndpoint, // Azure OpenAI Endpoint
                apiKey)      // Azure OpenAI Key
            .Build();        

Installed Packages Microsoft.SemanticKernel, 1.4.0 Microsoft.SemanticKernel.Connectors.OpenAI, 1.4.0 Microsoft.SemanticKernel.Planners.Handlebars, 1.4.0-preview System.Linq.Async, 6.0.1

Define/load plugin to facilitate use of chat history

In [3]:
using System.IO;

kernel.Plugins.Clear();

#pragma warning disable SKEXP0052
var pluginPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "plugins");
kernel.CreatePluginFromPromptDirectory(Path.Combine(pluginPath, "IdentityPlugin"));


Run

In [7]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner(new HandlebarsPlannerOptions() { AllowLoops = true });
var goal = "Application is for my own employees.";
var plan = await planner.CreatePlanAsync(kernel, goal);
Console.WriteLine("Plan:\n");
Console.WriteLine(plan.ToString());

var reloadedPlan = plan; //new HandlebarsPlan(plan.ToString());

var resp = await reloadedPlan.InvokeAsync(kernel);
Console.WriteLine("Result:\n");
Console.WriteLine(resp);

// Should be able to save and re-load a plan: https://devblogs.microsoft.com/semantic-kernel/migrating-from-the-sequential-and-stepwise-planners-to-the-new-handlebars-and-stepwise-planner/

Plan:

{{!-- Step 0: Extract necessary key values --}}
{{set "employeeList" employees}}

{{!-- Step 1: Create a table to display employee information --}}
<table>
  <thead>
    <tr>
      <th>Name</th>
      <th>Position</th>
      <th>Email</th>
    </tr>
  </thead>
  <tbody>
    {{!-- Step 2: Loop through each employee in the list --}}
    {{#each employeeList}}
    <tr>
      <td>{{name}}</td>
      <td>{{position}}</td>
      <td>{{email}}</td>
    </tr>
    {{/each}}
  </tbody>
</table>
Result:

<table>
  <thead>
    <tr>
      <th>Name</th>
      <th>Position</th>
      <th>Email</th>
    </tr>
  </thead>
  <tbody>
  </tbody>
</table>
